In [1]:
import shap
import numpy as np
import torch
import os

In [ ]:
# Retrieved saved model + re-open data train and test
is_cov = False

def load_graphs(input_dir, class_dict) :

    data, data_labels = [], [] # data containing the graphs and data_labels the associated seizure type labels
    i=0
    for szr_type in class_dict.keys() :
        szr_label = class_dict[szr_type]
        for _, _, files in os.walk(os.path.join(input_dir,szr_type)) :
            for npy_file in files :
                A = np.load(os.path.join(input_dir,szr_type,npy_file))

                # Normalise A (already normalised depending on the input)
                A = A/np.amax(A.flatten())
                if is_cov : 
                    L = torch.tensor(A).view(1,20,20)
                else : 
                    L = torch.tensor(np.diag(A*np.ones((A.shape[0],1)))-A).view(1,20,20)
                    #L = torch.tensor(A).view(1,20,20)

                data.append(L)
                data_labels.append(szr_label)

    return np.array(data), np.array(data_labels)

def train_test_data(input_dir, class_dict) :

    train, train_labels = load_graphs(os.path.join(input_dir,'train'), class_dict)
    test, test_labels = load_graphs(os.path.join(input_dir,'dev'), class_dict)

    return train, test, train_labels, test_labels

# Retrieve trained CNN
model = torch.load('low_lapl_50_CNN.pt')
model.eval()

input_dir = '../data/v1.5.2/graph_lapl_nolow'